Blackjack:
- One player against dealer.
- Player can stand or hit. (add split later if hand has same faces)
- Player can pick betting amount.
- Player has wallet tracked through game.
- Alert player of wins, losses, busts, ect.

Code Ideas/Description:
- Card class has face and suit with corresponding rank (ace set at 11 with logic to reduce to 1 when necessary).
- Deck class will hold deck. We will have a one deck, continuous shuffling game.
- Blackjack, 21, pays 2:1 (not yet implemented).
- Player cards are dealt face up. Dealer one card face down.
- Tie with dealer returns bet
- Dealer stands on 17+ (Later hit until hard 17+)


In [1]:
# Blackjack version 1
import random
from IPython.display import clear_output

#------------------------------------------Card Info----------------------------------------------------

faces = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
suits = ('Hearts', 'Spades', 'Clubs', 'Diamonds')
ranks = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
         'Queen':10, 'King':10, 'Ace':11}

#------------------------------------------Classes------------------------------------------------------

class Card:
    
    def __init__(self, face, suit):
        self.face = face
        self.suit = suit
        self.rank = ranks[face]
        
    def __str__(self):
        return f"{self.face} of {self.suit}"
    
class Deck:
    
    def __init__(self, cards = []):
        self.cards = cards
        
    def __str__(self):
        return f"I contain {len(self.cards)} cards."
    
    def shuffle(self):
        random.shuffle(self.cards)
        
    def add(self, card):
        self.cards.append(card)
        
    def deal(self):
        return self.cards.pop()
    
    def score(self):
        #Count aces
        ace_count = 0
        for card in self.cards:
            if card.face == "Ace":
                ace_count = ace_count+1
                
        #Compute hand total with aces as 11
        hand = 0
        for card in self.cards:
            hand = hand + card.rank
            
        #Adjust for aces if total is over 21
        while (ace_count > 0) and (hand > 21):
            hand = hand - 10
            ace_count = ace_count - 1
        
        return hand
    
class Gambler:
    
    def __init__(self, deck, bank):
        self.deck = deck
        self.bank = bank
        
    def __str__(self):
        return f"I have {self.bank} dollars."
        
    def bet(self, amount):
        self.bank = self.bank - amount
        return amount
    
    def win(self, amount):
        self.bank = self.bank + amount
        
    

#------------------------------------------Game Logic--------------------------------------------
print("Welcome to a 2:1 one deck continuous shuffling Blackjack game!")
print("Your dealer will stay on 17 and above.")
print("You have been given $100.")
print("Minimum bet is $5")
print("Have fun losing.")

move = None
while move not in ("1","2"):
    move = input("\nMake a selection:\n1 - Lets Play! \n2 - I'll pass.\n")
      
keep_playing = True
if move == "2":
      keep_playing = False
        
#The dealer is a gambler too... This will track house win/loss
dealer = Gambler(Deck([]), 0)

#Initialize player with $100 purse
player = Gambler(Deck([]),100)
      
while keep_playing:
    #Create the shoe, here we shuffle after every hand,"Continuous shuffling".
    shoe = Deck([])
    for suit in suits:
        for face in faces:
            shoe.add(Card(face,suit))
    shoe.shuffle()
    
    #Start dealer & player with empty hands
    player.deck = Deck([])
    dealer.deck = Deck([])
    
    #Deal initial hand
    dealer.deck.add(shoe.deal())
    player.deck.add(shoe.deal())
    dealer.deck.add(shoe.deal())
    player.deck.add(shoe.deal())
    
    #Prompt for bet
    bet = 0
    while bet<5 or bet>player.bank:
        bet = input(f"You have ${player.bank} \nEnter your bet, min = $5:\n$")
        
        #Error catch for non numerical input
        try:
            bet = float(bet)
        except ValueError:
            bet = 0
            print("Please enter a valid bet.")
            continue
        
        #Low bet return
        if bet<5:
            print("Please enter a valid bet.")
            continue
            
        #High bet return
        if bet>player.bank:
            print("You don't have that much money.\nPlease enter a valid bet.")
            continue
    
    #Ask player to hit until "no" or "bust"
    while player.deck.score() <= 21:
        #Start with clean output
        clear_output()
        
        #Show dealer's second card and players hand
        print("Dealer's Hand:")
        print("      -Hidden-")
        print("    " + dealer.deck.cards[1].__str__())
        print(f"Player's Hand({player.deck.score()}):")
        for card in player.deck.cards:
            print("    "+card.__str__())

        #Prompt 1-hit or 2-stand
        move = None
        while move not in ("1","2"):
            move = input("\nMake a selection:\n1 - Stand \n2 - Hit\n")    

        #Stay logic
        if move == "1":
            break
        
        #Hit logic
        if move == "2":
            player.deck.add(shoe.deal())
            #check for win/bust return to input
            
    #Dealer hit until 17+ (add ace count for soft 17 here.)
    while dealer.deck.score() < 17:
        dealer.deck.add(shoe.deal())

    #Show dealer and player hand along with winner
    clear_output()
    print(f"Dealer's Hand({dealer.deck.score()}):")
    for card in dealer.deck.cards:
        print("    "+card.__str__())
    print(f"Player's Hand({player.deck.score()}):")
    for card in player.deck.cards:
        print("    "+card.__str__())

    if player.deck.score() > 21:
        print(f"\nYou Bust!!!\nYou Lose ${bet}!!!")
        player.bank = player.bank - bet
        dealer.bank = dealer.bank + bet
    elif dealer.deck.score() > 21:
        print(f"\nDealer Bust!!!\nYou Win ${bet}!!!")
        player.bank = player.bank + bet
        dealer.bank = dealer.bank - bet
    elif dealer.deck.score() > player.deck.score():
        print(f"\nYou Lost ${bet}!!!")
        player.bank = player.bank - bet
        dealer.bank = dealer.bank + bet
    elif dealer.deck.score() < player.deck.score():
        print(f"\nYou Won ${bet}!!!")
    else:
        print("\nDraw!!!")

    #Play new hand?
    play_on = None
    print(f"You have ${player.bank}")
    
    #Kick out the broke
    if player.bank <5:
        print("Loooooser!")
        break
    
    while play_on not in ("1","2"):
        play_on = input("\nPlay On?\n1-Y\n2-N\n").upper()
    keep_playing = (play_on == "1")

Dealer's Hand(22):
    Four of Hearts
    Six of Diamonds
    Four of Clubs
    Eight of Clubs
Player's Hand(22):
    Five of Spades
    Seven of Diamonds
    Ten of Hearts

You Bust!!!
You Lose $100.0!!!
You have $0.0
Loooooser!
